In [1]:
import pandas as pd
import numpy as np
import mat73
from matplotlib import pyplot as plt
import scipy.signal as signal
import math

In [2]:
# Import the matlab files
data_dict = mat73.loadmat(r'C:/Users/keshe/AllEEG.mat')

In [3]:
# Parse the different fields of the structure
data_dict['AllEEG'].keys()

dict_keys(['AdEEG', 'BDMEEG', 'SubID', 'Triggers'])

In [4]:
# Parse the different fields of the structure
bdm = data_dict['AllEEG'].BDMEEG
ad = data_dict['AllEEG'].AdEEG
subid = data_dict['AllEEG'].SubID
triggers = data_dict['AllEEG'].Triggers

In [5]:
np.shape(bdm[0]['EEG'][0])

(8, 2250)

In [6]:
# Parse BDM data
bdm_eeg = []
bdm_cat = []
bdm_item = []
bdm_label = []
bdm_repitition = []
bdm_sub_id = []
for i in range(len(bdm)):
    sub_id = subid[i]
    for j in range(50):
        bdm_eeg.append(bdm[i]['EEG'][j])
        bdm_cat.append(bdm[i]['Category'][j])
        bdm_item.append(bdm[i]['Item'][j])
        bdm_label.append(bdm[i]['Label'][j])
        bdm_repitition.append(bdm[i]['Repitition'][j])
        bdm_sub_id.append(sub_id)

In [7]:
# Construct BDM Dataframe
colnames = ['sub_id' ,'eeg','category','label','repitition','item']
bdm_df = pd.DataFrame(data=[bdm_sub_id, bdm_eeg, bdm_cat, bdm_label, bdm_repitition, bdm_item],
                     index=colnames).transpose()
bdm_df

,sub_id,eeg,category,label,repitition,item
0,201.0,"[[22.125034535927973, 17.653907708965182, 3.83...",Beverages,80.6,1.0,Neviot
1,201.0,"[[-10.193899819384981, 0.8766877479518658, 0.3...",Hygene,100.0,1.0,Zippersmallbag
2,201.0,"[[-16.752930810400265, -13.97356521714783, -12...",Hygene,100.0,1.0,Sano99
3,201.0,"[[2.929928196900284, -9.54256955794608, -13.21...",Breakfast,70.7,1.0,Branflakesextra
4,201.0,"[[14.294047113047942, 22.568015892354225, 10.6...",DryGoods,100.0,1.0,Prince
...,...,...,...,...,...,...
3595,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",Hygene,39.3,2.0,GelDeLin
3596,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",Hygene,86.7,2.0,Sano99
3597,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",Beverages,84.8,2.0,Neviot
3598,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",Breakfast,17.4,2.0,ChocolateSpread


In [8]:
# Parse Ad data
ad[0].keys()

dict_keys(['AdID', 'Category', 'EEG', 'Item', 'ItemID', 'Label', 'Liking', 'Repitition', 'nArtifacts'])

In [9]:
range(len(ad[0]['EEG']))

range(0, 96)

In [10]:
# Parse Ad data
ad_eeg = []
ad_id = []
ad_cat = []
ad_item = []
ad_label = []
ad_repitition = []
ad_item_id = []
ad_liking = []
ad_sub_id = []
for i in range(len(ad)):
    sub_id = subid[i]
    for j in range(len(ad[i]['EEG'])):
        ad_eeg.append(ad[i]['EEG'][j])
        ad_cat.append(ad[i]['Category'][j])
        ad_item.append(ad[i]['Item'][j])
        ad_label.append(ad[i]['Label'][j])
        ad_repitition.append(ad[i]['Repitition'][j])
        ad_id.append(ad[i]['AdID'][j])
        ad_item_id.append(ad[i]['ItemID'][j])
        ad_liking.append(ad[i]['Liking'][j])
        ad_sub_id.append(sub_id)

In [11]:
# Construct Ad Dataframe
colnames = ['sub_id','eeg','ad_id','category','item','label','repitition','item_id','liking']
ad_df = pd.DataFrame(data=[ad_sub_id, ad_eeg, ad_id, ad_cat, ad_item, ad_label, ad_repitition, ad_item_id, ad_liking],
                     index=colnames).transpose()
ad_df

,sub_id,eeg,ad_id,category,item,label,repitition,item_id,liking
0,201.0,"[[6.169504138131352, 1.4996781805581034, 0.882...",3033.0,DryGoods,Prince,99.5,1.0,303.0,7.0
1,201.0,"[[-9.480992144445933, -10.385721551568722, -8....",1025.0,Breakfast,Branflakesextra,50.8,1.0,102.0,5.0
2,201.0,"[[-29.822218351462798, -22.74021099428629, -19...",5031.0,Snacks,Parachocolate,49.8,1.0,503.0,5.6
3,201.0,"[[19.14256891544619, 21.445438236058163, 11.27...",2052.0,Beverages,Neviot,76.6,1.0,205.0,6.0
4,201.0,"[[2.211251428382117, 4.856250386404814, 6.4057...",3013.0,DryGoods,YadMordechai,78.4,1.0,301.0,6.4
...,...,...,...,...,...,...,...,...,...
6952,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",3045.0,DryGoods,HEINZ,67.8,4.0,304.0,4.0
6953,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",3041.0,DryGoods,HEINZ,69.9,5.0,304.0,4.3
6954,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",5052.0,Snacks,Fitness,65.4,5.0,505.0,1.2
6955,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",2006.0,Beverages,FuzeTeaGreen,75.9,3.0,200.0,4.9


In [12]:
nan_ind = []
for i in range(len(ad_df)):
    nan_count = 0
    for j in range(len(ad_df['eeg'][i])):
        if math.isnan(np.sum(ad_df.eeg[i][j])):
            nan_count += 1
    nan_ind.append(nan_count)

In [13]:
liking = ad_df['liking']
a = liking.isna()
b = []
for index in range(0,len(liking)):
    b.append(liking[index])
liking[486]
liking_nan = np.where(np.isnan(b))
f = np.asarray(liking_nan)

numpy.ndarray

In [14]:
nan_eeg_entries = np.argwhere(nan_ind).flatten()
type(nan_eeg_entries)

numpy.ndarray

In [90]:
ad_id = data['ad_id']
ad_id_list = []
for id in ad_id:
    ad_id_list.append(float(id))
sub_id = data['sub_id']
sub_id_list = []
for id in sub_id:
    sub_id_list.append(float(id))
duplicate_dict = {'sub':sub_id_list,'ad':ad_id_list}
duplicate_test = pd.DataFrame(data=duplicate_dict)
duplicates = duplicate_test.duplicated()
a = []
for index in range(0,len(duplicates)):
    if duplicates[index] == True:
        a.append(index)
a

[]

In [91]:
nans = np.concatenate((f,nan_eeg_entries,a),axis=None)
nans_set = set(nans)

In [93]:
# Remove nan eeg and likings entries from dataframe
data = ad_df.drop(nans_set)
data

,sub_id,eeg,ad_id,category,item,label,repitition,item_id,liking
0,201.0,"[[6.169504138131352, 1.4996781805581034, 0.882...",3033.0,DryGoods,Prince,99.5,1.0,303.0,7.0
1,201.0,"[[-9.480992144445933, -10.385721551568722, -8....",1025.0,Breakfast,Branflakesextra,50.8,1.0,102.0,5.0
2,201.0,"[[-29.822218351462798, -22.74021099428629, -19...",5031.0,Snacks,Parachocolate,49.8,1.0,503.0,5.6
3,201.0,"[[19.14256891544619, 21.445438236058163, 11.27...",2052.0,Beverages,Neviot,76.6,1.0,205.0,6.0
4,201.0,"[[2.211251428382117, 4.856250386404814, 6.4057...",3013.0,DryGoods,YadMordechai,78.4,1.0,301.0,6.4
...,...,...,...,...,...,...,...,...,...
6852,272.0,"[[-35.134740326497386, -30.147006241623345, -1...",1031.0,Breakfast,ChocolateSpread,50.1,3.0,103.0,4.1
6853,272.0,"[[-2.542308377613169, -4.847342430446311, 3.74...",2042.0,Beverages,CocaCola2L,0.0,3.0,204.0,3.5
6854,272.0,"[[-7.1544970052599215, -2.2188501995059777, 1....",3031.0,DryGoods,Prince,67.1,3.0,303.0,3.5
6855,272.0,"[[8.701023382352462, 5.707112046757462, 2.2009...",3033.0,DryGoods,Prince,66.4,4.0,303.0,5.0


In [101]:
print(len(data.query('liking > 5.5')))
print(len(data.query('liking < 1.5')))

905
1023


In [102]:
set(data.category)

{'Beverages', 'Breakfast', 'DryGoods', 'Hygene', 'Snacks'}

In [103]:
bdm_sorted = bdm_df.sort_values(by = 'item',
                                inplace = False) 
bdm_sorted

,sub_id,eeg,category,label,repitition,item
2967,261.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",Snacks,12.0,1.0,Baracke
2592,253.0,"[[23.93310496444433, 36.33314815688214, 26.321...",Snacks,72.0,2.0,Baracke
3572,273.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",Snacks,4.9,1.0,Baracke
2560,253.0,"[[51.83940359091858, 43.394048245849454, 35.23...",Snacks,87.3,1.0,Baracke
3136,264.0,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...",Snacks,44.7,2.0,Baracke
...,...,...,...,...,...,...
2225,246.0,"[[-4.921993935609088, -20.14413748530816, -14....",Hygene,60.8,2.0,Zippersmallbag
403,207.0,"[[-11.470262129194927, -11.71349065222121, -18...",Hygene,4.8,1.0,Zippersmallbag
2218,246.0,"[[-19.208696174101064, -16.345583215859648, -1...",Hygene,67.3,1.0,Zippersmallbag
499,210.0,"[[12.953755991812193, -1.8996461720486693, -27...",Hygene,98.5,2.0,Zippersmallbag


In [117]:
def hunt_eeg_nans(df):
    nan_ind = []
    for i in range(len(df)):
        nan_count = 0
        for j in range(len(df['eeg'][i])):
            if math.isnan(np.sum(ad_df.eeg[i][j])):
                nan_count += 1
        nan_ind.append(nan_count)
    nan_eeg_entries = np.argwhere(nan_ind).flatten()
    return nan_eeg_entries

In [118]:
def hunt_liking_nans(df,col):
    l_col = ad_df[col]
    b = []
    for index in range(0,len(l_col)):
        b.append(l_col[index])
    l_col = np.where(np.isnan(b))
    f = np.asarray(l_col)
    return f

In [119]:
def hunt_duplicates(df):
    ad_id = data['ad_id']
    ad_id_list = []
    for id in ad_id:
        ad_id_list.append(float(id))
    sub_id = data['sub_id']
    sub_id_list = []
    for id in sub_id:
        sub_id_list.append(float(id))
    duplicate_dict = {'sub':sub_id_list,'ad':ad_id_list}
    duplicate_test = pd.DataFrame(data=duplicate_dict)
    duplicates = duplicate_test.duplicated()
    duplicate_rows = []
    for index in range(0,len(duplicates)):
        if duplicates[index] == True:
            duplicate_rows.append(index)
    return duplicate_rows

In [120]:
def clean_data(df):
    nan_eeg_entries = hunt_eeg_nans(df)
    liking_nans = hunt_liking_nans(df)
    duplicate_rows = hunt_duplicates(df)
    nans = np.concatenate((liking_nans,nan_eeg_entries,duplicate_rows),axis=None)
    nans_set = set(nans)
    # Remove nan eeg and likings entries from dataframe
    data = df.drop(nans_set)
    return data

In [122]:
data1 = clean_data(bdm)
data1

TypeError: list indices must be integers or slices, not str

In [125]:
before_pref = bdm_sorted.drop(columns=['eeg' , 'category' , 'repitition'])
before_pref

,sub_id,label,item
2967,261.0,12.0,Baracke
2592,253.0,72.0,Baracke
3572,273.0,4.9,Baracke
2560,253.0,87.3,Baracke
3136,264.0,44.7,Baracke
...,...,...,...
2225,246.0,60.8,Zippersmallbag
403,207.0,4.8,Zippersmallbag
2218,246.0,67.3,Zippersmallbag
499,210.0,98.5,Zippersmallbag
